In [1]:
import Random
Random.seed!(0)
pwd()

"/home/bach/iskra"

In [2]:
include("problem/units_and_constants.jl")
include("problem/configuration.jl")
config = Config();

┌ Info: Precompiling ParticleInCell [top-level]
└ @ Base loading.jl:1260


In [3]:
nx = 20         # number of nodes in x direction
ny = 20         # number of nodes in y direction
ts = 500        # number of time steps
Δh = 5cm        # cell size
Δt = 20ns       # time step
Lx = nx*Δh      # domain length in x direction
Ly = ny*Δh      # domain length in y direction
nothing

In [4]:
xs, ys = 0m:Δh:Lx, 0m:Δh:Ly
sx, sv = [0 Lx; 0 Ly], [0 -.1Δh/Δt; 0 0]
e = create_kinetic_species("e-", 20_000,-1qe, 1me, 1);
γ = create_gamma_ionization_source(1/Δt, sx, sv);

In [5]:
import RegularGrid, FiniteDifferenceMethod, ParticleInCell
config.grid    = RegularGrid.create_uniform_grid(xs, ys)
config.cells   = RegularGrid.create_staggered_grid(config.grid)
config.solver  = FiniteDifferenceMethod.create_poisson_solver(config.grid, ε0)
config.pusher  = ParticleInCell.create_boris_pusher()
config.species = [e];

In [6]:
nx, ny = size(config.grid)
mx, my = size(config.cells)
εr  = ones(mx, my, 1)
bcs = zeros(Int8, nx, ny, 1)
bcs[ nx,  1, 1] = 1
bcs[ nx, ny, 1] = 2
set_permittivity(εr)
create_electrode(bcs .== 1, config; σ=1ε0);
create_electrode(bcs .== 2, config; fixed=true);

In [7]:
import Diagnostics

function ParticleInCell.after_loop(i, t, dt)
  f, it = Diagnostics.new_iteration("01_single_particle", i, t, dt)
  Diagnostics.save_diagnostic(it, "e-/positionOffset/x")
  Diagnostics.save_diagnostic(it, "e-/positionOffset/y")
  Diagnostics.save_diagnostic(it, "e-/positionOffset/z")
  Diagnostics.save_diagnostic(it, "e-/position")
  Diagnostics.save_diagnostic(it, "e-/momentum")
  Diagnostics.save_diagnostic(it, "e-/weighting")
  Diagnostics.save_diagnostic(it, "e-/charge")
  Diagnostics.save_diagnostic(it, "e-/mass")
  Diagnostics.save_diagnostic(it, "e-/id")
  Diagnostics.save_diagnostic(it, "rho")
  Diagnostics.save_diagnostic(it, "phi")
  Diagnostics.save_diagnostic(it, "E")
  close(f)
end

nothing

In [8]:
ParticleInCell.init(γ, e, Δt)
@time ParticleInCell.solve(config, Δt, ts)

New diagnostics registered: e-/id
New diagnostics registered: e-/mass
New diagnostics registered: e-/charge
New diagnostics registered: e-/weighting
New diagnostics registered: e-/momentum
New diagnostics registered: e-/position
New diagnostics registered: e-/positionOffset/x
New diagnostics registered: e-/positionOffset/y
New diagnostics registered: e-/positionOffset/z
New diagnostics registered: e-/density
New diagnostics registered: rho
New diagnostics registered: phi
New diagnostics registered: E


ErrorException: type _GLOBAL_RNG has no field seed